# 🚀 LLM迎合性分析ランチャー - Google Colab版

**📋 このノートブックの目的**

Google Colab環境でLLMの迎合性（Sycophancy）分析を効率的に実行するためのランチャーです。

## 🎯 主要な改善点

1. **環境構築と実験実行の分離**: 問題の切り分けが容易
2. **段階的依存関係インストール**: `requirements.txt`のハングアップ問題を解決
3. **Git sparse-checkout**: 必要なファイルのみ高速取得
4. **自動実験実行**: nbconvertによる実験ノートブック自動実行
5. **包括的デバッグ支援**: 問題発生時の迅速な特定と解決

## 🔧 使用方法

1. **上から順番に各セルを実行**してください
2. **エラーが発生した場合**は該当セクションのトラブルシューティングを参照
3. **実験設定を変更したい場合**は対応するセクションで設定を調整

## ⚠️ 重要な注意事項

- このノートブックは**Google Colab環境専用**に最適化されています
- 各セルは**上から順番に実行**してください
- **データセットを事前にGoogle Driveに配置**してください（詳細は下記参照）
- **問題が発生した場合は焦らず**、該当セクションのデバッグ機能を活用してください

## 📁 データセット準備（実行前に必要）

**実行前に以下の準備を完了してください：**

1. **Google Driveに`eval_dataset`フォルダを作成**
2. **以下のファイルをアップロード：**
   - `are_you_sure.jsonl` (主要データセット - 必須)
   - `answer.jsonl`, `feedback.jsonl`, `few_shot_examples.jsonl` (オプション)
3. **推奨配置場所：** `/content/drive/MyDrive/eval_dataset/`

**注意：** データセットがない場合、セル2.5で詳細な配置手順が表示されます。

---

**🎉 準備ができたら下のセルから実行を開始してください！**

In [ ]:
# 🔧 セル1: 環境構築システムの設計
# ==============================================

import os
import sys
import warnings
import subprocess
from datetime import datetime

# 📋 実験設定（ここで一元管理）
# ==============================================
REPO_URL = "https://github.com/Itsuki966/my_sae.git"
REPO_DIR = "my_sae"
EXPERIMENT_BRANCH = "main"  # 使用するブランチ
TARGET_NOTEBOOK = "sycophancy_analysis_colab.ipynb"

# 実験メタデータ
EXPERIMENT_SESSION = {
    'session_id': datetime.now().strftime("%Y%m%d_%H%M%S"),
    'start_time': datetime.now().isoformat(),
    'branch': EXPERIMENT_BRANCH,
    'colab_detected': False,
    'gpu_available': False,
    'environment_status': 'initializing'
}

print("🚀 LLM迎合性分析ランチャー - 環境構築開始")
print("=" * 60)

# Google Colab環境の検出
def detect_colab_environment():
    """Google Colab環境を検出し、基本情報を表示"""
    try:
        import google.colab
        print("✅ Google Colab環境を検出")
        EXPERIMENT_SESSION['colab_detected'] = True
        
        # ランタイム情報の表示
        print("📊 Colab環境情報:")
        print(f"   🔖 セッションID: {EXPERIMENT_SESSION['session_id']}")
        print(f"   🕐 開始時刻: {EXPERIMENT_SESSION['start_time']}")
        
        return True
    except ImportError:
        print("⚠️ ローカル環境での実行を検出")
        print("💡 このノートブックはGoogle Colab用に最適化されています")
        EXPERIMENT_SESSION['colab_detected'] = False
        return False

# GPU環境の確認
def check_gpu_environment():
    """GPU環境の確認と表示"""
    try:
        import torch
        
        if torch.cuda.is_available():
            gpu_name = torch.cuda.get_device_name(0)
            gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
            gpu_count = torch.cuda.device_count()
            
            print("🚀 GPU環境:")
            print(f"   📱 GPU名: {gpu_name}")
            print(f"   💾 VRAM: {gpu_memory:.1f}GB")
            print(f"   🔢 GPU数: {gpu_count}")
            
            EXPERIMENT_SESSION['gpu_available'] = True
            EXPERIMENT_SESSION['gpu_name'] = gpu_name
            EXPERIMENT_SESSION['gpu_memory_gb'] = round(gpu_memory, 1)
            
            # T4 GPU特化の最適化ヒント
            if "T4" in gpu_name:
                print("🎯 T4 GPU検出: Gemma-2Bモデルが推奨されます")
            
            return True
        else:
            print("❌ GPU利用不可 - CPUモードで実行されます")
            print("⚠️ 実行時間が大幅に増加する可能性があります")
            EXPERIMENT_SESSION['gpu_available'] = False
            return False
            
    except ImportError:
        print("❌ PyTorchが利用できません")
        return False

# 基本環境変数の設定
def setup_environment_variables():
    """メモリ効率化のための環境変数設定"""
    print("🔧 環境変数を設定中...")
    
    # PyTorchメモリ管理最適化  
    os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
    os.environ['TOKENIZERS_PARALLELISM'] = 'false'
    
    # 警告の抑制
    warnings.filterwarnings('ignore')
    
    print("✅ 環境変数設定完了")

# メイン環境検出の実行
print("🔍 環境検出を開始...")

colab_detected = detect_colab_environment()
gpu_available = check_gpu_environment()
setup_environment_variables()

EXPERIMENT_SESSION['environment_status'] = 'configured'

print(f"\n📋 環境構築サマリー:")
print(f"   🌐 Colab環境: {'✅' if colab_detected else '❌'}")
print(f"   🚀 GPU利用: {'✅' if gpu_available else '❌'}")
print(f"   📁 作業ディレクトリ: {REPO_DIR}")
print(f"   🌿 対象ブランチ: {EXPERIMENT_BRANCH}")
print(f"   📓 実行予定ノートブック: {TARGET_NOTEBOOK}")

print(f"\n💡 次のセル (セル2) でリポジトリ取得を実行してください")
print("=" * 60)

In [ ]:
# 📂 セル2: リポジトリ管理とsparse-checkout実装
# ==============================================

def execute_git_sparse_checkout():
    """Git sparse-checkoutを使用してリポジトリから必要ファイルのみを取得"""
    
    print("📂 Git sparse-checkoutを開始...")
    
    # 既存ディレクトリの確認とクリーンアップ
    if os.path.exists(REPO_DIR):
        print(f"🧹 既存の{REPO_DIR}ディレクトリを削除中...")
        !rm -rf {REPO_DIR}
    
    try:
        # ステップ1: 骨格のみクローン（高速）
        print("📥 リポジトリ骨格をクローン中...")
        !git clone --filter=blob:none --no-checkout -b {EXPERIMENT_BRANCH} {REPO_URL} {REPO_DIR}
        
        # 作業ディレクトリに移動
        original_dir = os.getcwd()
        os.chdir(REPO_DIR)
        
        # ステップ2: sparse-checkoutの初期化
        print("🔧 sparse-checkoutを初期化中...")
        !git sparse-checkout init --cone
        
        # ステップ3: 必要ファイルの指定（eval_datasetはGoogle Driveから取得）
        required_files = [
            "sycophancy_analyzer.py",
            "config.py", 
            "memory_optimizer.py",
            "sycophancy_analysis_colab.ipynb",
            "requirements.txt",
            "pyproject.toml"
        ]
        
        print("📋 必要ファイルを指定中...")
        for file in required_files:
            print(f"   🔸 {file}")
        
        # sparse-checkoutの設定
        files_spec = " ".join(required_files)
        !git sparse-checkout set {files_spec}
        
        # ステップ4: 実際のチェックアウト実行
        print("📋 ファイルをチェックアウト中...")
        !git checkout {EXPERIMENT_BRANCH}
        
        # コミット情報の取得
        commit_info = !git log --oneline -1
        commit_hash = !git rev-parse --short HEAD
        
        EXPERIMENT_SESSION['commit_hash'] = commit_hash[0] if commit_hash else 'unknown'
        EXPERIMENT_SESSION['commit_info'] = commit_info[0] if commit_info else 'unknown'
        
        # 元のディレクトリに戻る
        os.chdir(original_dir)
        
        # 取得結果の確認
        print(f"\n✅ リポジトリ取得完了:")
        print(f"   📌 コミット: {EXPERIMENT_SESSION['commit_hash']}")
        print(f"   📝 {EXPERIMENT_SESSION['commit_info']}")
        
        # 取得ファイルの確認
        print(f"\n📁 取得ファイル確認:")
        for file in required_files:
            file_path = os.path.join(REPO_DIR, file)
            if os.path.exists(file_path):
                if os.path.isfile(file_path):
                    size_kb = os.path.getsize(file_path) / 1024
                    print(f"   ✅ {file} ({size_kb:.1f}KB)")
                else:
                    # ディレクトリの場合
                    try:
                        file_count = len([f for f in os.listdir(file_path) 
                                        if os.path.isfile(os.path.join(file_path, f))])
                        print(f"   ✅ {file}/ ({file_count}ファイル)")
                    except:
                        print(f"   ✅ {file}/ (ディレクトリ)")
            else:
                print(f"   ❌ {file} (見つかりません)")
        
        # パスにリポジトリディレクトリを追加
        repo_path = os.path.abspath(REPO_DIR)
        if repo_path not in sys.path:
            sys.path.insert(0, repo_path)
            print(f"📁 Pythonパスに追加: {repo_path}")
        
        EXPERIMENT_SESSION['repository_status'] = 'ready'
        return True
        
    except Exception as e:
        print(f"❌ リポジトリ取得エラー: {e}")
        print(f"💡 以下を確認してください:")
        print(f"   - ネットワーク接続")
        print(f"   - ブランチ'{EXPERIMENT_BRANCH}'の存在")
        print(f"   - リポジトリURL: {REPO_URL}")
        
        EXPERIMENT_SESSION['repository_status'] = 'failed'
        EXPERIMENT_SESSION['repository_error'] = str(e)
        return False

def check_repository_integrity():
    """取得したリポジトリの整合性確認"""
    
    print("🔍 リポジトリ整合性チェック...")
    
    critical_files = ["sycophancy_analyzer.py", "config.py"]
    missing_files = []
    
    for file in critical_files:
        file_path = os.path.join(REPO_DIR, file)
        if not os.path.exists(file_path):
            missing_files.append(file)
    
    if missing_files:
        print(f"❌ 重要ファイルが不足:")
        for file in missing_files:
            print(f"   - {file}")
        return False
    else:
        print("✅ 重要ファイルの整合性確認完了")
        return True

# メインリポジトリ取得処理の実行
print("📂 リポジトリ管理を開始...")
print("=" * 60)

success = execute_git_sparse_checkout()

if success:
    integrity_ok = check_repository_integrity()
    
    if integrity_ok:
        print(f"\n🎉 リポジトリ準備完了!")
        print(f"💡 次のセル (セル3) で依存関係インストールを実行してください")
    else:
        print(f"\n⚠️ リポジトリに問題があります")
        print(f"💡 セル5のデバッグ機能を使用して問題を特定してください")
else:
    print(f"\n❌ リポジトリ取得に失敗しました")
    print(f"💡 セル5のデバッグ機能を使用してください")

print("=" * 60)

In [ ]:
# 📁 セル2.5: Google Driveマウントとデータセット準備
# ==============================================

def mount_google_drive():
    """Google Driveをマウントしてデータセットへのアクセスを準備"""
    
    print("📁 Google Driveマウントを開始...")
    
    try:
        from google.colab import drive
        
        # Google Driveマウント実行
        print("🔒 Google認証を開始...")
        print("💡 ブラウザでGoogleアカウントにログインしてください")
        
        drive.mount('/content/drive')
        
        # マウント成功確認
        drive_path = '/content/drive/MyDrive'
        if os.path.exists(drive_path):
            print("✅ Google Driveマウント成功")
            
            # ドライブ内容の確認（最初のレベルのみ）
            try:
                drive_contents = os.listdir(drive_path)
                print(f"📋 マイドライブ内容 ({len(drive_contents)}項目):")
                for item in drive_contents[:10]:  # 最初の10項目のみ表示
                    item_path = os.path.join(drive_path, item)
                    if os.path.isdir(item_path):
                        print(f"   📁 {item}/")
                    else:
                        print(f"   📄 {item}")
                if len(drive_contents) > 10:
                    print(f"   ... および {len(drive_contents) - 10} 個の追加項目")
            except Exception as e:
                print(f"⚠️ ドライブ内容の確認に失敗: {e}")
            
            EXPERIMENT_SESSION['google_drive_mounted'] = True
            EXPERIMENT_SESSION['drive_path'] = drive_path
            return True
        else:
            print("❌ Google Driveマウントポイントが見つかりません")
            return False
            
    except ImportError:
        print("❌ google.colabモジュールが利用できません")
        print("💡 このセルはGoogle Colab環境でのみ動作します")
        return False
    except Exception as e:
        print(f"❌ Google Driveマウントエラー: {e}")
        return False

def setup_dataset_from_drive():
    """Google Driveからデータセットを設定"""
    
    print("📊 データセット設定を開始...")
    
    # データセットの推奨パス（ユーザーが調整可能）
    possible_dataset_paths = [
        '/content/drive/MyDrive/sae_datasets/eval_dataset',
        '/content/drive/MyDrive/research/sae/eval_dataset', 
        '/content/drive/MyDrive/eval_dataset',
        '/content/drive/MyDrive/Colab Notebooks/eval_dataset',
        '/content/drive/MyDrive/sae/eval_dataset'
    ]
    
    print("🔍 以下のパスでデータセットを検索中:")
    for path in possible_dataset_paths:
        print(f"   📁 {path}")
    
    dataset_path = None
    found_files = []
    
    # データセットディレクトリの検索
    for candidate_path in possible_dataset_paths:
        if os.path.exists(candidate_path):
            print(f"✅ データセットディレクトリを発見: {candidate_path}")
            
            # ディレクトリ内容の確認
            try:
                files = [f for f in os.listdir(candidate_path) 
                        if f.endswith('.jsonl') or f.endswith('.json')]
                if files:
                    dataset_path = candidate_path
                    found_files = files
                    print(f"📋 データファイル ({len(files)}個):")
                    for file in files:
                        file_path = os.path.join(candidate_path, file)
                        if os.path.exists(file_path):
                            size_kb = os.path.getsize(file_path) / 1024
                            print(f"   📄 {file} ({size_kb:.1f}KB)")
                    break
            except Exception as e:
                print(f"⚠️ ディレクトリ確認エラー: {e}")
                continue
    
    if not dataset_path:
        print("❌ データセットが見つかりませんでした")
        print("\n💡 手動でデータセットを配置してください:")
        print("   1. Google Driveに 'eval_dataset' フォルダを作成")
        print("   2. 以下のファイルをアップロード:")
        print("      - are_you_sure.jsonl (主要データセット)")
        print("      - answer.jsonl, feedback.jsonl, few_shot_examples.jsonl")
        print("   3. 推奨パス: /content/drive/MyDrive/eval_dataset/")
        print("\n🔄 データセット配置後、このセルを再実行してください")
        
        EXPERIMENT_SESSION['dataset_status'] = 'not_found'
        return False
    
    # リポジトリ内のeval_datasetにシンボリックリンクを作成
    repo_dataset_path = os.path.join(REPO_DIR, 'eval_dataset')
    
    try:
        # 既存のディレクトリやリンクがあれば削除
        if os.path.exists(repo_dataset_path):
            if os.path.islink(repo_dataset_path):
                os.unlink(repo_dataset_path)
            else:
                import shutil
                shutil.rmtree(repo_dataset_path)
        
        # シンボリックリンクを作成
        os.symlink(dataset_path, repo_dataset_path)
        
        print(f"✅ データセットリンク作成成功:")
        print(f"   📂 {repo_dataset_path} -> {dataset_path}")
        
        # リンクの検証
        if os.path.exists(repo_dataset_path):
            linked_files = os.listdir(repo_dataset_path)
            print(f"🔗 リンク確認: {len(linked_files)}ファイル利用可能")
            
            EXPERIMENT_SESSION['dataset_status'] = 'ready'
            EXPERIMENT_SESSION['dataset_path'] = dataset_path
            EXPERIMENT_SESSION['dataset_files'] = found_files
            return True
        else:
            print("❌ シンボリックリンクの作成に失敗")
            return False
            
    except Exception as e:
        print(f"❌ データセットリンク作成エラー: {e}")
        print("💡 フォールバック: 直接コピーを試行中...")
        
        # フォールバック: ファイルを直接コピー
        try:
            import shutil
            os.makedirs(repo_dataset_path, exist_ok=True)
            
            copied_files = 0
            for file in found_files:
                src = os.path.join(dataset_path, file)
                dst = os.path.join(repo_dataset_path, file)
                shutil.copy2(src, dst)
                copied_files += 1
                print(f"   📄 {file} をコピー")
            
            print(f"✅ フォールバック成功: {copied_files}ファイルをコピー")
            
            EXPERIMENT_SESSION['dataset_status'] = 'ready_copied'
            EXPERIMENT_SESSION['dataset_path'] = repo_dataset_path
            EXPERIMENT_SESSION['dataset_files'] = found_files
            return True
            
        except Exception as copy_error:
            print(f"❌ フォールバックコピーも失敗: {copy_error}")
            EXPERIMENT_SESSION['dataset_status'] = 'failed'
            return False

def verify_dataset_integrity():
    """データセットの整合性確認"""
    
    print("🔍 データセット整合性チェック...")
    
    repo_dataset_path = os.path.join(REPO_DIR, 'eval_dataset')
    
    if not os.path.exists(repo_dataset_path):
        print("❌ eval_datasetディレクトリが存在しません")
        return False
    
    # 重要ファイルの確認
    critical_files = ['are_you_sure.jsonl']  # 最重要ファイル
    optional_files = ['answer.jsonl', 'feedback.jsonl', 'few_shot_examples.jsonl']
    
    missing_critical = []
    missing_optional = []
    
    for file in critical_files:
        file_path = os.path.join(repo_dataset_path, file)
        if not os.path.exists(file_path):
            missing_critical.append(file)
        else:
            # ファイルサイズとサンプル数の確認
            try:
                size_mb = os.path.getsize(file_path) / (1024 * 1024)
                print(f"✅ {file} ({size_mb:.1f}MB)")
                
                # JSONLファイルの行数確認
                if file.endswith('.jsonl'):
                    with open(file_path, 'r', encoding='utf-8') as f:
                        lines = sum(1 for line in f if line.strip())
                    print(f"   📊 {lines}件のデータサンプル")
                    
            except Exception as e:
                print(f"⚠️ {file} 確認エラー: {e}")
    
    for file in optional_files:
        file_path = os.path.join(repo_dataset_path, file)
        if not os.path.exists(file_path):
            missing_optional.append(file)
        else:
            size_kb = os.path.getsize(file_path) / 1024
            print(f"✅ {file} ({size_kb:.1f}KB)")
    
    if missing_critical:
        print(f"❌ 重要ファイルが不足: {', '.join(missing_critical)}")
        return False
    
    if missing_optional:
        print(f"⚠️ オプションファイルが不足: {', '.join(missing_optional)}")
        print("💡 これらのファイルがなくても基本的な分析は実行可能です")
    
    print("✅ データセット整合性確認完了")
    return True

# メインデータセット準備処理
print("📁 Google Driveデータセット準備を開始...")
print("=" * 60)

# Google Driveのマウント
mount_success = mount_google_drive()

if mount_success:
    # データセットの設定
    dataset_success = setup_dataset_from_drive()
    
    if dataset_success:
        # データセット整合性確認
        integrity_ok = verify_dataset_integrity()
        
        if integrity_ok:
            print(f"\n🎉 データセット準備完了!")
            print(f"📊 分析で使用されるデータセット: {EXPERIMENT_SESSION.get('dataset_path', 'unknown')}")
            print(f"📋 利用可能ファイル: {len(EXPERIMENT_SESSION.get('dataset_files', []))}個")
            print(f"💡 次のセル (セル3) で依存関係インストールを実行してください")
        else:
            print(f"\n⚠️ データセットに問題があります")
            print(f"💡 上記の指示に従ってデータセットを確認してください")
    else:
        print(f"\n❌ データセット設定に失敗しました")
        print(f"💡 上記の指示に従ってGoogle Driveにデータセットを配置してください")
else:
    print(f"\n❌ Google Driveマウントに失敗しました")
    print(f"💡 Colabの認証画面で適切にログインしてください")

print("=" * 60)

In [ ]:
# 📦 セル3: 段階的依存関係インストール
# ==============================================

import time
import subprocess

def install_package_safely(package_name, description="", timeout=300):
    """個別パッケージの安全なインストール"""
    print(f"📦 {description or package_name} をインストール中...")
    
    start_time = time.time()
    try:
        # quietモードでインストール
        result = subprocess.run(
            ['pip', 'install', '-q', package_name], 
            timeout=timeout,
            capture_output=True,
            text=True
        )
        
        elapsed = time.time() - start_time
        
        if result.returncode == 0:
            print(f"   ✅ {package_name} ({elapsed:.1f}s)")
            return True
        else:
            print(f"   ❌ {package_name} - エラー: {result.stderr[:100]}...")
            return False
            
    except subprocess.TimeoutExpired:
        print(f"   ⏰ {package_name} - タイムアウト ({timeout}s)")
        return False
    except Exception as e:
        print(f"   ❌ {package_name} - 例外: {e}")
        return False

def install_core_dependencies():
    """コア依存関係の段階的インストール"""
    
    print("🔧 コア依存関係のインストール開始...")
    
    # フェーズ1: 基本ライブラリ
    print("\n📋 フェーズ1: 基本ライブラリ")
    basic_packages = [
        ("torch", "PyTorch (GPU対応)"),
        ("transformers", "HuggingFace Transformers"),
        ("accelerate", "メモリ効率化ライブラリ"),
    ]
    
    failed_basic = []
    for package, desc in basic_packages:
        if not install_package_safely(package, desc, timeout=600):  # 10分タイムアウト
            failed_basic.append(package)
    
    if failed_basic:
        print(f"⚠️ 基本ライブラリのインストールに失敗: {', '.join(failed_basic)}")
        return False
    
    # フェーズ2: 分析ライブラリ
    print("\n📋 フェーズ2: 分析ライブラリ")
    analysis_packages = [
        ("plotly", "可視化ライブラリ"),
        ("tqdm", "プログレスバー"),
        ("numpy", "数値計算"),
        ("pandas", "データ処理")
    ]
    
    failed_analysis = []
    for package, desc in analysis_packages:
        if not install_package_safely(package, desc, timeout=300):  # 5分タイムアウト
            failed_analysis.append(package)
    
    # フェーズ3: SAE Lens（最も重要）
    print("\n📋 フェーズ3: SAE Lens（重要）")
    if not install_package_safely("sae-lens", "SAE Lens（核心ライブラリ）", timeout=900):  # 15分タイムアウト
        print("❌ SAE Lensのインストールに失敗 - これは致命的です")
        return False
    
    if failed_analysis:
        print(f"⚠️ 一部の分析ライブラリが失敗しましたが、続行可能です: {', '.join(failed_analysis)}")
    
    return True

def install_from_requirements():
    """requirements.txtからの一括インストール（フォールバック）"""
    
    requirements_path = os.path.join(REPO_DIR, "requirements.txt")
    
    if not os.path.exists(requirements_path):
        print(f"❌ requirements.txtが見つかりません: {requirements_path}")
        return False
    
    print("📋 requirements.txtから一括インストールを試行...")
    print("⚠️ この処理は時間がかかる場合があります...")
    
    try:
        start_time = time.time()
        
        # 最大30分のタイムアウト
        result = subprocess.run(
            ['pip', 'install', '-q', '-r', requirements_path],
            timeout=1800,  # 30分
            capture_output=True,
            text=True
        )
        
        elapsed = time.time() - start_time
        
        if result.returncode == 0:
            print(f"✅ requirements.txt一括インストール完了 ({elapsed:.1f}s)")
            return True
        else:
            print(f"❌ requirements.txt一括インストール失敗")
            if result.stderr:
                print(f"エラー詳細: {result.stderr[:200]}...")
            return False
            
    except subprocess.TimeoutExpired:
        print("⏰ requirements.txt一括インストール - タイムアウト (30分)")
        print("💡 段階的インストールに切り替えてください")
        return False
    except Exception as e:
        print(f"❌ requirements.txtインストール例外: {e}")
        return False

def verify_installation():
    """インストール結果の検証"""
    
    print("🔍 インストール結果を検証中...")
    
    required_modules = {
        'torch': 'PyTorch',
        'transformers': 'HuggingFace Transformers',
        'sae_lens': 'SAE Lens',
        'plotly': 'Plotly',
        'tqdm': 'tqdm'
    }
    
    missing_modules = []
    working_modules = []
    
    for module, name in required_modules.items():
        try:
            __import__(module)
            working_modules.append(name)
            print(f"   ✅ {name}")
        except ImportError:
            missing_modules.append(name)
            print(f"   ❌ {name}")
    
    if missing_modules:
        print(f"\n⚠️ 不足モジュール: {', '.join(missing_modules)}")
        print(f"💡 セル5の手動インストール機能を使用してください")
        return False
    else:
        print(f"\n✅ 全モジュール検証完了 ({len(working_modules)}個)")
        return True

# メイン依存関係インストール処理
print("📦 依存関係インストールを開始...")
print("=" * 60)

# インストール戦略の選択
INSTALL_STRATEGY = "staged"  # "staged" or "bulk"

if INSTALL_STRATEGY == "staged":
    print("🎯 段階的インストール戦略を選択")
    success = install_core_dependencies()
elif INSTALL_STRATEGY == "bulk":
    print("🎯 一括インストール戦略を選択") 
    success = install_from_requirements()
else:
    print("❌ 不明なインストール戦略")
    success = False

# インストール結果の検証
if success:
    verification_success = verify_installation()
    
    if verification_success:
        print(f"\n🎉 依存関係インストール完了!")
        print(f"💡 次のセル (セル4) で実験ノートブック実行を開始してください")
        EXPERIMENT_SESSION['dependencies_status'] = 'ready'
    else:
        print(f"\n⚠️ 一部のモジュールに問題があります")
        print(f"💡 セル5のデバッグ機能で詳細確認してください")
        EXPERIMENT_SESSION['dependencies_status'] = 'partial'
else:
    print(f"\n❌ 依存関係インストールに失敗しました")
    print(f"💡 セル5の手動インストール機能を試してください")
    EXPERIMENT_SESSION['dependencies_status'] = 'failed'

print("=" * 60)

In [ ]:
# 🔬 セル4: 実験ノートブック実行システム（改良版）
# ==============================================

import json
import shlex
from datetime import datetime

def setup_jupyter_environment():
    """Jupyter実行環境の設定"""
    print("🔧 Jupyter実行環境を設定中...")
    
    # Python デバッガ警告の抑制
    os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'
    os.environ['PYTHONFROZEN'] = '0'
    
    # Jupyter設定の最適化
    os.environ['JUPYTER_PLATFORM_DIRS'] = '1'
    
    print("✅ Jupyter環境設定完了")

def execute_experiment_notebook_direct():
    """実験ノートブックを直接実行（nbconvertの代替）"""
    
    notebook_path = os.path.join(REPO_DIR, TARGET_NOTEBOOK)
    
    if not os.path.exists(notebook_path):
        print(f"❌ 実験ノートブックが見つかりません: {notebook_path}")
        return False
    
    print(f"🔬 実験ノートブック直接実行開始: {TARGET_NOTEBOOK}")
    print("=" * 50)
    
    # 実行メタデータ
    execution_start = datetime.now()
    
    try:
        # Jupyter kernel直接実行を試行
        print("⚡ Jupyter kernel直接実行中...")
        print("⏱️ この処理には数分〜十数分かかる場合があります...")
        
        # 実行ディレクトリを変更
        original_dir = os.getcwd()
        os.chdir(REPO_DIR)
        
        # デバッグ情報の出力
        print(f"📁 作業ディレクトリ: {os.getcwd()}")
        print(f"🔍 ターゲットノートブック: {TARGET_NOTEBOOK}")
        print(f"✅ ファイル存在確認: {os.path.exists(TARGET_NOTEBOOK)}")
        
        # nbconvertコマンドを環境変数付きで実行
        execute_command = [
            'python', '-Xfrozen_modules=off', '-m', 'nbconvert',
            '--to', 'notebook',
            '--execute', 
            '--inplace',
            '--allow-errors',
            '--log-level=ERROR',  # ログレベルを下げて警告を抑制
            '--ExecutePreprocessor.timeout=2400',  # 40分タイムアウト
            '--ExecutePreprocessor.kernel_name=python3',
            TARGET_NOTEBOOK
        ]
        
        print(f"🔍 実行コマンド: {' '.join(execute_command)}")
        
        # 環境変数を設定して実行
        env = os.environ.copy()
        env['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'
        env['PYTHONFROZEN'] = '0'
        
        # 実行ログをキャプチャ
        result = subprocess.run(
            execute_command,
            capture_output=True,
            text=True,
            timeout=2400,  # 40分タイムアウト
            env=env
        )
        
        # 元のディレクトリに戻る
        os.chdir(original_dir)
        
        execution_end = datetime.now()
        execution_time = (execution_end - execution_start).total_seconds()
        
        if result.returncode == 0:
            print(f"✅ 実験ノートブック実行完了!")
            print(f"⏱️ 実行時間: {execution_time:.1f}秒 ({execution_time/60:.1f}分)")
            
            # 実行結果の保存
            EXPERIMENT_SESSION['notebook_execution'] = {
                'status': 'success',
                'execution_time_seconds': execution_time,
                'start_time': execution_start.isoformat(),
                'end_time': execution_end.isoformat(),
                'method': 'direct_nbconvert'
            }
            
            return True
        else:
            print(f"❌ 実験ノートブック実行エラー")
            print(f"💡 エラーコード: {result.returncode}")
            
            # エラー出力の詳細表示（デバッガ警告をフィルタリング）
            if result.stderr:
                error_lines = result.stderr.split('\n')
                filtered_errors = [line for line in error_lines 
                                 if not ('Debugger warning' in line or 'frozen modules' in line)]
                if filtered_errors:
                    print(f"🔍 重要なエラー詳細:")
                    for line in filtered_errors[:10]:  # 最初の10行のみ
                        if line.strip():
                            print(f"   {line}")
            
            if result.stdout:
                print(f"📋 実行出力（抜粋）:")
                stdout_lines = result.stdout.split('\n')
                for line in stdout_lines[-5:]:  # 最後の5行のみ
                    if line.strip():
                        print(f"   {line}")
            
            EXPERIMENT_SESSION['notebook_execution'] = {
                'status': 'failed',
                'error': result.stderr if result.stderr else 'Unknown error',
                'stdout': result.stdout if result.stdout else '',
                'returncode': result.returncode,
                'execution_time_seconds': execution_time,
                'method': 'direct_nbconvert'
            }
            
            return False
            
    except subprocess.TimeoutExpired:
        print("⏰ 実験ノートブック実行 - タイムアウト (40分)")
        os.chdir(original_dir)  # ディレクトリを元に戻す
        EXPERIMENT_SESSION['notebook_execution'] = {
            'status': 'timeout',
            'timeout_seconds': 2400,
            'method': 'direct_nbconvert'
        }
        return False
    except Exception as e:
        print(f"❌ 実験ノートブック実行例外: {e}")
        os.chdir(original_dir)  # ディレクトリを元に戻す
        EXPERIMENT_SESSION['notebook_execution'] = {
            'status': 'exception',
            'error': str(e),
            'method': 'direct_nbconvert'
        }
        return False

def execute_experiment_notebook_alternative():
    """代替手段による実験実行（Python直接実行）"""
    
    print("🔄 代替実行方法を試行中...")
    
    # Python スクリプトとして直接実行を試行
    analyzer_path = os.path.join(REPO_DIR, "sycophancy_analyzer.py")
    
    if not os.path.exists(analyzer_path):
        print(f"❌ 分析スクリプトが見つかりません: {analyzer_path}")
        return False
    
    try:
        original_dir = os.getcwd()
        os.chdir(REPO_DIR)
        
        print("🐍 Python直接実行を試行中...")
        
        # Python環境で直接実行
        execute_command = [
            'python', '-Xfrozen_modules=off', 'sycophancy_analyzer.py'
        ]
        
        env = os.environ.copy()
        env['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'
        
        result = subprocess.run(
            execute_command,
            capture_output=True,
            text=True,
            timeout=1800,  # 30分タイムアウト
            env=env
        )
        
        os.chdir(original_dir)
        
        if result.returncode == 0:
            print("✅ Python直接実行成功!")
            EXPERIMENT_SESSION['notebook_execution'] = {
                'status': 'success',
                'method': 'python_direct'
            }
            return True
        else:
            print(f"❌ Python直接実行失敗: {result.stderr}")
            return False
            
    except Exception as e:
        print(f"❌ 代替実行例外: {e}")
        os.chdir(original_dir)
        return False

def check_experiment_results():
    """実験結果の確認"""
    
    print("🔍 実験結果を確認中...")
    
    results_dir = os.path.join(REPO_DIR, "results")
    plots_dir = os.path.join(REPO_DIR, "plots")
    
    found_results = []
    
    # 結果ファイルの確認
    if os.path.exists(results_dir):
        result_files = [f for f in os.listdir(results_dir) if f.endswith('.json')]
        if result_files:
            print(f"📊 結果ファイル ({len(result_files)}個):")
            for file in result_files[:3]:  # 最初の3個のみ表示
                file_path = os.path.join(results_dir, file)
                size_kb = os.path.getsize(file_path) / 1024
                print(f"   📄 {file} ({size_kb:.1f}KB)")
                found_results.append(file)
    
    # プロットファイルの確認
    if os.path.exists(plots_dir):
        plot_files = [f for f in os.listdir(plots_dir) if f.endswith('.html')]
        if plot_files:
            print(f"📈 可視化ファイル ({len(plot_files)}個):")
            for file in plot_files[:3]:  # 最初の3個のみ表示
                print(f"   📊 {file}")
                found_results.append(file)
    
    return len(found_results) > 0

def save_experiment_log():
    """実験ログの保存"""
    
    print("📋 実験ログを保存中...")
    
    # 実験セッション完了時刻の記録
    EXPERIMENT_SESSION['end_time'] = datetime.now().isoformat()
    EXPERIMENT_SESSION['total_duration_seconds'] = (
        datetime.fromisoformat(EXPERIMENT_SESSION['end_time']) - 
        datetime.fromisoformat(EXPERIMENT_SESSION['start_time'])
    ).total_seconds()
    
    # ログファイル名
    log_filename = f"experiment_log_{EXPERIMENT_SESSION['session_id']}.json"
    log_path = os.path.join(REPO_DIR, log_filename)
    
    try:
        with open(log_path, 'w', encoding='utf-8') as f:
            json.dump(EXPERIMENT_SESSION, f, indent=2, ensure_ascii=False)
        
        print(f"✅ 実験ログ保存: {log_filename}")
        return True
    except Exception as e:
        print(f"❌ ログ保存エラー: {e}")
        return False

def display_experiment_summary():
    """実験サマリーの表示"""
    
    print("\n" + "=" * 60)
    print("🎉 実験完了サマリー")
    print("=" * 60)
    
    print(f"📋 実験メタデータ:")
    print(f"   🔖 セッションID: {EXPERIMENT_SESSION['session_id']}")
    print(f"   🌿 ブランチ: {EXPERIMENT_SESSION['branch']}")
    print(f"   📌 コミット: {EXPERIMENT_SESSION.get('commit_hash', 'unknown')}")
    print(f"   🕐 開始時刻: {EXPERIMENT_SESSION['start_time']}")
    print(f"   🕐 終了時刻: {EXPERIMENT_SESSION.get('end_time', 'unknown')}")
    
    if 'total_duration_seconds' in EXPERIMENT_SESSION:
        duration = EXPERIMENT_SESSION['total_duration_seconds']
        print(f"   ⏱️ 総実行時間: {duration:.1f}秒 ({duration/60:.1f}分)")
    
    # 各段階のステータス
    print(f"\n📊 各段階の実行状況:")
    print(f"   🔧 環境構築: {EXPERIMENT_SESSION.get('environment_status', 'unknown')}")
    print(f"   📂 リポジトリ: {EXPERIMENT_SESSION.get('repository_status', 'unknown')}")
    print(f"   📁 データセット: {EXPERIMENT_SESSION.get('dataset_status', 'unknown')}")
    print(f"   📦 依存関係: {EXPERIMENT_SESSION.get('dependencies_status', 'unknown')}")
    
    if 'notebook_execution' in EXPERIMENT_SESSION:
        nb_status = EXPERIMENT_SESSION['notebook_execution']['status']
        nb_method = EXPERIMENT_SESSION['notebook_execution'].get('method', 'unknown')
        print(f"   🔬 実験実行: {nb_status} ({nb_method})")
        
        if nb_status == 'success' and 'execution_time_seconds' in EXPERIMENT_SESSION['notebook_execution']:
            nb_time = EXPERIMENT_SESSION['notebook_execution']['execution_time_seconds']
            print(f"      ⏱️ 実験時間: {nb_time:.1f}秒 ({nb_time/60:.1f}分)")
    
    # GPU使用状況
    if EXPERIMENT_SESSION.get('gpu_available'):
        print(f"   🚀 GPU: {EXPERIMENT_SESSION.get('gpu_name', 'unknown')} "
              f"({EXPERIMENT_SESSION.get('gpu_memory_gb', 'unknown')}GB)")

# メイン実験実行処理
print("🔬 実験ノートブック実行システム（改良版）を開始...")
print("=" * 60)

# 前提条件の確認
if EXPERIMENT_SESSION.get('dependencies_status') not in ['ready', 'partial']:
    print("❌ 依存関係が準備されていません")
    print("💡 セル3で依存関係のインストールを完了してください")
elif EXPERIMENT_SESSION.get('dataset_status') not in ['ready', 'ready_copied']:
    print("❌ データセットが準備されていません")
    print("💡 セル2.5でデータセットの準備を完了してください")
else:
    # Jupyter環境設定
    setup_jupyter_environment()
    
    # 実験実行（メイン手法）
    print("\n🎯 メイン実行手法: 改良されたnbconvert")
    execution_success = execute_experiment_notebook_direct()
    
    # メイン手法が失敗した場合の代替実行
    if not execution_success:
        print("\n🔄 代替実行手法を試行...")
        execution_success = execute_experiment_notebook_alternative()
    
    if execution_success:
        # 結果確認
        results_found = check_experiment_results()
        
        if results_found:
            print("✅ 実験結果ファイルを確認しました")
        else:
            print("⚠️ 実験結果ファイルが見つかりません")
            print("💡 実験は完了しましたが、結果ファイルの生成に問題がある可能性があります")
        
        # ログ保存
        save_experiment_log()
        
        # サマリー表示
        display_experiment_summary()
        
        print(f"\n🎉 実験完了!")
        print(f"💡 結果は {REPO_DIR}/results/ ディレクトリを確認してください")
        print(f"📊 可視化結果は {REPO_DIR}/plots/ ディレクトリにあります")
        
    else:
        print("❌ 全ての実験実行手法が失敗しました")
        print("💡 以下を確認してください:")
        print("   1. セル5のデバッグ機能で詳細確認")
        print("   2. 依存関係の再インストール（セル3）")
        print("   3. データセットの確認（セル2.5）")
        
        # エラー詳細を表示
        if 'notebook_execution' in EXPERIMENT_SESSION:
            nb_exec = EXPERIMENT_SESSION['notebook_execution']
            if 'error' in nb_exec:
                print(f"\n🔍 最新エラー情報:")
                error_text = nb_exec['error']
                # 重要なエラーのみ表示（デバッガ警告を除く）
                error_lines = error_text.split('\n') if error_text else []
                important_errors = [line for line in error_lines[:10] 
                                  if line.strip() and not ('Debugger warning' in line or 'frozen modules' in line)]
                for line in important_errors:
                    print(f"   {line}")
                    
        # メモリクリア（次回実行のため）
        print(f"\n🧹 メモリクリアを実行中...")
        try:
            import gc
            gc.collect()
            if 'torch' in sys.modules:
                import torch
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
            print("✅ メモリクリア完了")
        except:
            print("⚠️ メモリクリアに問題がありました")

print("=" * 60)

In [ ]:
# 🔧 セル5: デバッグとトラブルシューティング
# ==============================================

def debug_environment():
    """環境デバッグ情報の表示"""
    
    print("🔍 環境デバッグ情報")
    print("-" * 40)
    
    # Python環境
    print(f"Python version: {sys.version}")
    print(f"Current directory: {os.getcwd()}")
    print(f"Python path: {sys.path[:3]}...")  # 最初の3つのみ表示
    
    # Google Colab環境
    try:
        import google.colab
        print("Google Colab: ✅ 利用可能")
        
        # GPU情報
        try:
            import torch
            if torch.cuda.is_available():
                print(f"GPU: ✅ {torch.cuda.get_device_name(0)}")
                print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / (1024**3):.1f}GB")
                
                # メモリ使用状況
                allocated = torch.cuda.memory_allocated(0) / (1024**3)
                reserved = torch.cuda.memory_reserved(0) / (1024**3)
                print(f"GPU Memory - Allocated: {allocated:.2f}GB, Reserved: {reserved:.2f}GB")
            else:
                print("GPU: ❌ 利用不可")
        except ImportError:
            print("PyTorch: ❌ インストールされていません")
            
    except ImportError:
        print("Google Colab: ❌ ローカル環境")
    
    # ディスク容量
    import shutil
    total, used, free = shutil.disk_usage("/")
    print(f"Disk - Total: {total//(1024**3)}GB, Free: {free//(1024**3)}GB")

def debug_repository_status():
    """リポジトリ状態のデバッグ"""
    
    print("📂 リポジトリデバッグ情報")
    print("-" * 40)
    
    if os.path.exists(REPO_DIR):
        print(f"✅ リポジトリディレクトリ存在: {REPO_DIR}")
        
        # Git情報
        original_dir = os.getcwd()
        try:
            os.chdir(REPO_DIR)
            
            # ブランチ確認
            branch_info = !git branch --show-current
            print(f"Current branch: {branch_info[0] if branch_info else 'unknown'}")
            
            # コミット情報
            commit_info = !git log --oneline -1
            print(f"Latest commit: {commit_info[0] if commit_info else 'unknown'}")
            
            # sparse-checkout状態
            sparse_info = !git sparse-checkout list
            if sparse_info:
                print(f"Sparse-checkout files: {len(sparse_info)} items")
                for item in sparse_info[:5]:  # 最初の5つのみ表示
                    print(f"  - {item}")
            
            os.chdir(original_dir)
            
        except Exception as e:
            print(f"Git情報取得エラー: {e}")
            os.chdir(original_dir)
        
        # ファイル確認
        print("\nファイル確認:")
        required_files = ["sycophancy_analyzer.py", "config.py", TARGET_NOTEBOOK]
        for file in required_files:
            file_path = os.path.join(REPO_DIR, file)
            if os.path.exists(file_path):
                size = os.path.getsize(file_path) / 1024
                print(f"  ✅ {file} ({size:.1f}KB)")
            else:
                print(f"  ❌ {file} (不存在)")
    else:
        print(f"❌ リポジトリディレクトリが存在しません: {REPO_DIR}")

def debug_dependencies():
    """依存関係のデバッグ"""
    
    print("📦 依存関係デバッグ情報")
    print("-" * 40)
    
    critical_modules = {
        'torch': 'PyTorch',
        'transformers': 'HuggingFace Transformers', 
        'sae_lens': 'SAE Lens',
        'accelerate': 'Accelerate',
        'plotly': 'Plotly',
        'tqdm': 'tqdm',
        'numpy': 'NumPy',
        'pandas': 'Pandas'
    }
    
    print("モジュール確認:")
    for module, name in critical_modules.items():
        try:
            imported_module = __import__(module)
            version = getattr(imported_module, '__version__', 'unknown')
            print(f"  ✅ {name}: v{version}")
        except ImportError:
            print(f"  ❌ {name}: インストールされていません")
        except Exception as e:
            print(f"  ⚠️ {name}: エラー ({e})")

def manual_install_dependencies():
    """手動での依存関係インストール"""
    
    print("🔧 手動依存関係インストール")
    print("-" * 40)
    
    critical_packages = [
        "torch",
        "transformers", 
        "sae-lens",
        "accelerate",
        "plotly",
        "tqdm"
    ]
    
    print("重要パッケージを個別にインストール中...")
    
    for package in critical_packages:
        print(f"\n📦 {package} をインストール中...")
        try:
            result = !pip install -q {package}
            print(f"  ✅ {package} インストール完了")
        except Exception as e:
            print(f"  ❌ {package} インストールエラー: {e}")
    
    print(f"\n🔍 インストール結果を確認...")
    debug_dependencies()

def clear_memory():
    """メモリクリア"""
    
    print("🧹 メモリクリア実行中...")
    
    # Python garbage collection
    import gc
    collected = gc.collect()
    print(f"  🗑️ Python GC: {collected} オブジェクト回収")
    
    # GPU メモリクリア
    try:
        import torch
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.cuda.ipc_collect()
            
            allocated = torch.cuda.memory_allocated(0) / (1024**3)
            reserved = torch.cuda.memory_reserved(0) / (1024**3)
            print(f"  🚀 GPU Memory: {allocated:.2f}GB allocated, {reserved:.2f}GB reserved")
    except ImportError:
        print("  ⚠️ PyTorch利用不可 - GPU メモリクリアスキップ")

def display_experiment_session():
    """実験セッション情報の表示"""
    
    print("📋 実験セッション情報")
    print("-" * 40)
    
    if 'EXPERIMENT_SESSION' in globals():
        for key, value in EXPERIMENT_SESSION.items():
            if isinstance(value, dict):
                print(f"{key}:")
                for sub_key, sub_value in value.items():
                    print(f"  {sub_key}: {sub_value}")
            else:
                print(f"{key}: {value}")
    else:
        print("実験セッション情報がありません")

def run_full_diagnostic():
    """完全診断の実行"""
    
    print("🔬 完全診断を実行中...")
    print("=" * 60)
    
    debug_environment()
    print()
    debug_repository_status() 
    print()
    debug_dependencies()
    print()
    display_experiment_session()
    print()
    clear_memory()
    
    print("=" * 60)
    print("✅ 完全診断完了")

# デバッグ機能メニュー
print("🔧 デバッグとトラブルシューティング")
print("=" * 60)

print("利用可能なデバッグ機能:")
print("1. run_full_diagnostic() - 完全システム診断")
print("2. debug_environment() - 環境情報確認")
print("3. debug_repository_status() - リポジトリ状態確認")
print("4. debug_dependencies() - 依存関係確認")
print("5. manual_install_dependencies() - 手動依存関係インストール")
print("6. clear_memory() - メモリクリア")
print("7. display_experiment_session() - 実験セッション情報表示")

print(f"\n💡 使用方法:")
print(f"   問題が発生した場合は、まず run_full_diagnostic() を実行してください")
print(f"   特定の問題に対しては個別の診断機能を使用してください")

print(f"\n🚨 よくある問題と解決策:")
print(f"   📦 依存関係エラー → manual_install_dependencies()")
print(f"   💾 メモリ不足 → clear_memory()")
print(f"   📂 ファイル不足 → debug_repository_status()")
print(f"   🐍 環境問題 → debug_environment()")

print("=" * 60)

# 自動的に基本診断を実行
print("🔍 基本診断を自動実行中...")
debug_environment()
print()
if os.path.exists(REPO_DIR):
    debug_repository_status()
print("✅ 基本診断完了")